# Thresholds WaterStress 

* Purpose of script: calculate total demand (Dom, IrrLinear, Liv, Ind) and Reduced Runoff and water stress.
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20171004

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M10D04 UTC 14:47


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
S3_INPUT_PATH =  "s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/"

INPUT_FILE_NAME = "Y2017M09D15_RH_Add_Basin_Data_V02"

EC2_INPUT_PATH = "/volumes/data/Y2017M10D04_RH_Threshold_WaterStress_V02/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M10D04_RH_Threshold_WaterStress_V02/output"



In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

rm: cannot remove '/volumes/data/Y2017M10D04_RH_Threshold_WaterStress_V02/input': No such file or directory
rm: cannot remove '/volumes/data/Y2017M10D04_RH_Threshold_WaterStress_V02/output': No such file or directory


In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V02.pkl to ../../../../data/Y2017M10D04_RH_Threshold_WaterStress_V02/input/Y2017M09D15_RH_Add_Basin_Data_V02.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D15_RH_Add_Basin_Data_V01/output/Y2017M09D15_RH_Add_Basin_Data_V02.csv to ../../../../data/Y2017M10D04_RH_Threshold_WaterStress_V02/input/Y2017M09D15_RH_Add_Basin_Data_V02.csv


In [6]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:00:13.669310
